# Solving the Lorenz System Using Physics-Informed Neural Networks (PINNs)

## Problem Statement

The **Lorenz system** is a classic example of a 3-variable chaotic ODE system originally derived from simplified convection rolls in the atmosphere. It is defined by

$$
\begin{aligned}
\dot x &= \sigma\,(y - x),\\
\dot y &= x\,(\rho - z) - y,\\
\dot z &= x\,y - \beta\,z,
\end{aligned}
\qquad t \ge 0
$$

Where:  
- \(x(t), y(t), z(t)\) are the state variables.  
- \($\sigma$\), \($\rho$\), and \($\beta$\) are system parameters.  
- We enforce the **initial condition**  
  $$
  (x(0),\,y(0),\,z(0)) = (x_0,\,y_0,\,z_0).
  $$

Typical parameter values for the canonical “Lorenz attractor” are:  
- \($\sigma$ = 10\)  
- \($\rho$ = 28\)  
- \($\beta$ = $\tfrac{8}{3}$\)  

A common choice of initial state is \((x_0,y_0,z_0) = (1,1,1)\).

---

## Chaotic Dynamics

- No closed-form solution exists for arbitrary \(t\).  
- Exhibits **sensitive dependence on initial conditions** (chaos).  
- Trajectories for \($\rho$ > 24.74\) converge to the famous **Lorenz attractor**.

---

## PINN Objective

We approximate each state by a neural network  
\[
(\,\hat x(t;\theta),\;\hat y(t;\theta),\;\hat z(t;\theta)\,)
\]
and train to minimize the combined loss

$$
\mathcal{L}
=
\underbrace{\mathrm{MSE}_{\rm res}}_{\text{ODE residuals}}
\;+\;
\underbrace{\mathrm{MSE}_{\rm ic}}_{\text{initial conditions}}.
$$

- **Residual loss** (enforces the Lorenz ODEs at residual points \(t_i\)):  
  $$
  \mathrm{MSE}_{\rm res}
  = \frac{1}{N_r} \sum_{i=1}^{N_r}
  \Bigl[
    \dot{\hat x}(t_i) - \sigma\bigl(\hat y(t_i)-\hat x(t_i)\bigr)
  \Bigr]^2
  + 
  \Bigl[
    \dot{\hat y}(t_i) - \bigl(\hat x(t_i)\,(\rho-\hat z(t_i)) - \hat y(t_i)\bigr)
  \Bigr]^2
  + 
  \Bigl[
    \dot{\hat z}(t_i) - \bigl(\hat x(t_i)\,\hat y(t_i) - \beta\,\hat z(t_i)\bigr)
  \Bigr]^2.
  $$

- **Initial condition loss**:  
  $$
  \mathrm{MSE}_{\rm ic}
  = (\hat x(0)-x_0)^2
  + (\hat y(0)-y_0)^2
  + (\hat z(0)-z_0)^2.
  $$

---

## Why Use PINNs?

- **Mesh-free, continuous surrogate**: yields smooth \(\hat x(t)\), \(\hat y(t)\), \(\hat z(t)\) for all \(t\).  
- **Physics-aware loss**: directly enforces the ODE system, even in chaotic regimes.  
- **Automatic differentiation**: computes \(\dot{\hat x}\), \(\dot{\hat y}\), \(\dot{\hat z}\) easily.  
- **Flexible sampling**: choose residual points anywhere in \([0,T]\) to capture rapid divergence.

Let’s now define our network architecture, sample residual and IC points, and train the PINN to reproduce the Lorenz attractor trajectories!  


In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 3)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
def ode(t, x, y, z, sigma, rho, beta):
    t = t.clone().detach().requires_grad_(True)
    out = network(t)      # [N,3]
    x = out[:, 0:1]       
    y = out[:, 1:2]
    z = out[:, 2:3]

    dx_dt = torch.autograd.grad(x, t, grad_outputs=torch.ones_like(x), create_graph=True)[0]
    dy_dt = torch.autograd.grad(y, t, grad_outputs=torch.ones_like(y), create_graph=True)[0]
    dz_dt = torch.autograd.grad(z, t, grad_outputs=torch.ones_like(z), create_graph=True)[0]

    f_x = sigma * (y_t - x_t)
    f_y = (x * (rho - z)) - y
    f_z = (x_t * y_t) - (beta)

    res_x = dx_dt - f_x
    res_y = dy_dt - f_y
    res_z = dz_dt - f_z

    return res_x, res_y, res_z

In [ ]:
def loss(network, t, x, y, z, t0, x0, y0, z0, sigma, rho, beta):
    # ODE residual loss
    res_x, res_y, res_z = ode(t, x, y, z, sigma, rho, beta)
    MSE_res = torch.mean(res_x**2) + torch.mean(res_y**2) + torch.mean(res_z**2)

    # initial-condition loss
    T0_pred = network(t0)      # [N,3]
    x0_pred = T0_pred[:, 0:1]       
    y0_pred = T0_pred[:, 1:2]
    z0_pred = T0_pred[:, 2:3]
    MSE_ic = torch.mean((x0_pred  - x0)**2) + torch.mean((y0_pred  - y0)**2) + torch.mean((z0_pred  - z0)**2)

    return MSE_res + MSE_ic

In [ ]:
def train(network, t, x, y, z, t0, x0, y0, z0, sigma, rho, beta, epochs, lr = 1e-3):
    optimizer = torch.optim.Adam(network.parameters(), lr=lr)
    loss_list = []

    for epoch in range(1, epochs + 1):
        optimizer.zero_grad()
        loss_value = loss(network, t, x, y, z, t0, x0, y0, z0, sigma, rho, beta)
        loss_list.append(loss_value.item())
        loss_value.backward()
        optimizer.step()

        if epoch % 100 == 0 or epoch == 1:
            print(f"Epoch {epoch}/{epochs} — Loss: {loss_value.item():.3e}")

    return loss_list